In [87]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import random
from collections import Counter
from sklearn.metrics import roc_curve, auc, average_precision_score
path = 'steam-200k.csv'
df = pd.read_csv(path, header = None,
                 names = ['UserID', 'Game', 'Action', 'Hours', 'Not Needed'])
df['Hours_Played'] = df['Hours'].astype('float32')
df.loc[(df['Action'] == 'purchase') & (df['Hours'] == 1.0), 'Hours_Played'] = 0
df.UserID = df.UserID.astype('int')
#df = df.sort_values(['UserID', 'Game', 'Hours_Played'])
clean_df = df.drop_duplicates(['UserID', 'Game'], keep = 'last').drop(['Action', 'Hours', 'Not Needed'], axis = 1)
clean_df = clean_df.reset_index(drop=True)
clean_df

,UserID,Game,Hours_Played
0,151603712,The Elder Scrolls V Skyrim,273.000000
1,151603712,Fallout 4,87.000000
2,151603712,Spore,14.900000
3,151603712,Fallout New Vegas,12.100000
4,151603712,Left 4 Dead 2,8.900000
5,151603712,HuniePop,8.500000
6,151603712,Path of Exile,8.100000
7,151603712,Poly Bridge,7.500000
8,151603712,Left 4 Dead,3.300000
9,151603712,Team Fortress 2,2.800000


In [88]:
df_g=clean_df.groupby(by=['UserID']).max()
df_g.to_csv('groupUserID.csv',index = True, header = True, encoding = 'utf-8')

In [14]:
df_g

,Game,Hours_Played
UserID,,
5250,Team Fortress Classic,144.0
76767,Worms Armageddon,365.0
86540,XCOM Enemy Unknown,113.0
103360,Team Fortress Classic,0.0
144736,Team Fortress Classic,0.1
181212,Team Fortress Classic,1.8
229911,Worms Reloaded,165.0
298950,XCOM Enemy Within,1019.0
299153,Team Fortress Classic,0.0


In [76]:
path = 'gameid.csv'
gameid = pd.read_csv(path, header = None)
gameid

,0,1
0,GID,Game
1,1,007 Legends
2,2,0RBITALIS
3,3,1... 2... 3... KICK IT! (Drop That Beat Like a...
4,4,10 Second Ninja
5,5,"10,000,000"
6,6,100% Orange Juice
7,7,1000 Amps
8,8,12 Labours of Hercules
9,9,12 Labours of Hercules II The Cretan Bull


In [77]:
path = 'NewID.csv'
userid = pd.read_csv(path, header = None)
userid

,0,1
0,1,5250
1,2,76767
2,3,86540
3,4,103360
4,5,144736
5,6,181212
6,7,229911
7,8,298950
8,9,299153
9,10,381543


In [81]:
userid[1].astype('int64')


0             5250
1            76767
2            86540
3           103360
4           144736
5           181212
6           229911
7           298950
8           299153
9           381543
10          547685
11          554278
12          561758
13          577614
14          604988
15          622362
16          635733
17          683019
18          714122
19          748719
20          835015
21          861238
22          871990
23          937567
24          948368
25          975449
26          976129
27          983600
28          994489
29         1006880
           ...    
12363    308894258
12364    308925132
12365    308925147
12366    308940605
12367    308954355
12368    308970742
12369    308971716
12370    308997600
12371    309038666
12372    309052991
12373    309058572
12374    309107542
12375    309138595
12376    309167186
12377    309181805
12378    309188905
12379    309213952
12380    309216884
12381    309228590
12382    309255941
12383    309262440
12384    309

In [89]:
for i in range(12393):
    for j in range(12393):
        if clean_df.loc[i,'UserID'] == userid.loc[j,1]:
            clean_df.loc[i,'UserID'] = userid.loc[j,0]

In [90]:
for i in range(12393):
    for j in range(5156):
        if clean_df.loc[i,'Game'] == gameid.loc[j,1]:
            clean_df.loc[i,'Game'] = gameid.loc[j,0]

In [67]:
int(userid.loc[5,1])


144736

In [66]:
clean_df.loc[7,'UserID'] 

151603712

In [91]:
clean_df

,UserID,Game,Hours_Played
0,5495,4365,273.000000
1,5495,1679,87.000000
2,5495,3998,14.900000
3,5495,1680,12.100000
4,5495,2476,8.900000
5,5495,2194,8.500000
6,5495,3115,8.100000
7,5495,3214,7.500000
8,5495,2475,3.300000
9,5495,4258,2.800000
